In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from collections import defaultdict
from datetime import timedelta

def parse_time(time_str):
    if not time_str or time_str.strip() in ['Tiempo de Finales', 'DQ', '', 'Puntos']:
        return None
    time_str = time_str.replace('x', '').replace('X', '').strip()
    try:
        if ':' in time_str:
            parts = time_str.replace(',', '.').split(':')
            if len(parts) == 3:  # Format: hh:mm:ss.ms
                return timedelta(hours=float(parts[0]), minutes=float(parts[1]), seconds=float(parts[2]))
            elif len(parts) == 2:  # Format: mm:ss.ms
                return timedelta(minutes=float(parts[0]), seconds=float(parts[1]))
        elif ',' in time_str:
            seconds, milliseconds = time_str.split(',')
            return timedelta(seconds=float(seconds), milliseconds=float(milliseconds))
        else:
            return timedelta(seconds=float(time_str))
    except ValueError:
        return None

def process_event(event_data, output_dir):
    event_title = event_data[0].strip()
    print(f"\nProcessing {event_title}")
    
    times = []
    ages = []
    
    for line in event_data[1:]:
        parts = line.split(',')
        if len(parts) >= 5:
            name_age = parts[0].strip('"').split(',')
            if len(name_age) == 2:
                try:
                    age = int(name_age[1])
                    time = parse_time(parts[4])
                    if time:
                        times.append(time.total_seconds())
                        ages.append(age)
                except ValueError:
                    continue

    if not times:
        print(f"No valid data found for {event_title}")
        return

    print(f"Number of participants: {len(times)}")
    print(f"Average time: {timedelta(seconds=sum(times)/len(times))}")
    print(f"Fastest time: {timedelta(seconds=min(times))}")
    print(f"Slowest time: {timedelta(seconds=max(times))}")

    event_name = re.sub(r'[^\w\-_\. ]', '_', event_title)
    
    plt.figure(figsize=(12, 6))
    sns.histplot(times, kde=True)
    plt.title(f"Time Distribution - {event_title}")
    plt.xlabel("Time (seconds)")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{event_name}_distribution.png"))
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.scatter(ages, times)
    plt.title(f"Time vs Age - {event_title}")
    plt.xlabel("Age")
    plt.ylabel("Time (seconds)")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{event_name}_scatter.png"))
    plt.close()

def main(filename, output_dir, max_events=None):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    current_event = []
    event_count = 0

    with open(filename, 'r') as file:
        for line in file:
            if line.strip().startswith('Evento'):
                if current_event:
                    process_event(current_event, output_dir)
                    event_count += 1
                    if max_events and event_count >= max_events:
                        break
                current_event = [line]
            elif current_event:
                current_event.append(line)

    if current_event and (not max_events or event_count < max_events):
        process_event(current_event, output_dir)

# Sample usage
input_file = 'output.txt'
output_directory = 'swimming_analysis_output'
max_events_to_process = 10  # Set to None to process all events

if __name__ == "__main__":
    main(input_file, output_directory, max_events_to_process)


Processing Evento 2  Hombres 16-18 200 CC Metro Estilo Libre,,,,,
No valid data found for Evento 2  Hombres 16-18 200 CC Metro Estilo Libre,,,,,

Processing Evento 3  Niñas 12 Años de Edad 50 CC Metro Estilo de Pecho,,,,,
No valid data found for Evento 3  Niñas 12 Años de Edad 50 CC Metro Estilo de Pecho,,,,,

Processing Evento 3  Niñas 13 Años de Edad 50 CC Metro Estilo de Pecho,,,,,
No valid data found for Evento 3  Niñas 13 Años de Edad 50 CC Metro Estilo de Pecho,,,,,

Processing Evento 4  Niños 12 Años de Edad 50 CC Metro Estilo de Pecho,,,,,
No valid data found for Evento 4  Niños 12 Años de Edad 50 CC Metro Estilo de Pecho,,,,,

Processing Evento 4  Niños 12-13 50 CC Metro Estilo de Pecho,,,,,
No valid data found for Evento 4  Niños 12-13 50 CC Metro Estilo de Pecho,,,,,

Processing Evento 4  Niños 13 Años de Edad 50 CC Metro Estilo de Pecho,,,,,
No valid data found for Evento 4  Niños 13 Años de Edad 50 CC Metro Estilo de Pecho,,,,,

Processing Evento 5  Mujeres 19 & Mayores 1